# Invertible Deep Learning

## Advanced Invertible Layers
In this notebook we will demonstrate techniques for inverting a few other common layers in deep learning.

### Signed sin and arcsin
The "signed sin" function extends sin to negatively valued numbers in a mirrored fashion, since sin is only defined for positive values.

mathjax for signed sin

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = np.array([-4, -3, -2, -1, 0, 1, 2, 3, 4])
signed_sin_data = data * np.sin(np.abs(data)) * np.sign(data)

print(signed_sin_data)

inverted = np.arcsin((1/data) * signed_sin_data) * np.sign(signed_sin_data)
print(inverted)

[-3.02720998  0.42336002  1.81859485  0.84147098  0.          0.84147098
  1.81859485  0.42336002 -3.02720998]
[-0.85840735 -0.14159265 -1.14159265 -1.                 nan  1.
  1.14159265  0.14159265  0.85840735]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in multiply
  


In [ ]:

data = np.array([1, 2, 3, 4])
forward = data * np.tanh(data)

print(forward)

inverted = np.arctanh(forward/data) 
print(inverted)

[0.76159416 1.92805516 2.98516426 3.9973172 ]
[1. 2. 3. 4.]


In [ ]:

data = np.array([-1, -0.5, 0, 0.5, 1, 2, 3, 4])
forward = np.abs(data) * np.tanh(data)

print(forward)

inverted = np.arctanh(np.abs(forward)/np.abs(data))*np.sign(data) 
print(inverted)

[-0.76159416 -0.23105858  0.          0.23105858  0.76159416  1.92805516
  2.98516426  3.9973172 ]
[-1.  -0.5  nan  0.5  1.   2.   3.   4. ]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


Add a filter for 0, and recode in tensorflow, and you've got a pair of invertible activation functions. Tanh was popular in earlier research, but its only current usage is in LSTM/GRU, and I am not going to invert *those*.

## Parametric ReLU (Monotonic variant)

A more powerful version of LeakyReLU is Parametric ReLU, which has a learnable vector of alpha values for all hidden neurons. This can be used to modulate feature maps in Convolutional networks. We will present a simplified variant and its inverse.

The problem with Parametric ReLU as published (in the same paper that introduced He kernel initialization!) is that the alpha values are allowed to be negative. This means that the function is not monotonic. It is easy to alter Parametric ReLU to be monotonic, with the simple application of abs(). Here is the Keras source for Parametric ReLU:

```
    def call(self, inputs):
        pos = tf.keras.backend.relu(inputs)
        neg = -self.alpha * tf.keras.backend.relu(-inputs)
        return pos + neg
```

This change forces monotonicity:
```
    def call(self, inputs):
        pos = tf.keras.backend.relu(inputs)
        neg = -tf.math.abs(self.alpha) * tf.keras.backend.relu(-inputs)
        return pos + neg
```

To invert this layer, we need to capture the layer object in another layer and refer to it, inverting the alpha values:
```
    def call(self, inputs):
        pos = tf.keras.backend.relu(inputs)
        neg = -tf.math.abs(1/tf.math.abs(self.other_layer.alpha)) * tf.keras.backend.relu(-inputs)
        return pos + neg
```

Taking the abs() value of the alpha vector violates the probability theory behind weight initialization (Glorot, He, etc) but it seems to work fine.

In [ ]:

class MonotonicPReLU(tf.keras.layers.Layer):
    """Monotonic Parametric Rectified Linear Unit.
    It follows:
    ```
      f(x) = alpha * x for x < 0
      f(x) = x for x >= 0
    ```
    where `alpha` is a learned array with the same shape as x.
    To achieve monotonicity, force learned alpha params above 0 via abs().
    Input shape:
      Arbitrary. Use the keyword argument `input_shape`
      (tuple of integers, does not include the samples axis)
      when using this layer as the first layer in a model.
    Output shape:
      Same shape as the input.
    """

    def __init__(
        self,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.supports_masking = True

    def build(self, input_shape):
        param_shape = list(input_shape[1:])
        self.alpha = self.add_weight(
            shape=param_shape,
            name="alpha",
            initializer='glorot_uniform'
        )
        self.input_spec = tf.keras.layers.InputSpec(ndim=len(input_shape), axes={})
        self.built = True

    def call(self, inputs):
        pos = tf.keras.backend.relu(inputs)
        neg = -tf.math.abs(self.alpha) * tf.keras.backend.relu(-inputs)
        return pos + neg

    def compute_output_shape(self, input_shape):
        return input_shape

    
class InvertedMonotonicPReLU(tf.keras.layers.Layer):
    """Inverted match of Monotonic Parametric Rectified Linear Unit.
        Supplies inverted version of given layer.
    """

    def __init__(
        self,
        other_layer,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.other_layer = other_layer
        self.supports_masking = True

    def build(self, input_shape):
        param_shape = list(input_shape[1:])
        self.input_spec = tf.keras.layers.InputSpec(ndim=len(input_shape), axes={})
        self.params = []
        self.built = True

    def call(self, inputs):
        pos = tf.keras.backend.relu(inputs)
        neg = -(1/tf.math.abs(self.other_layer.alpha)) * tf.keras.backend.relu(-inputs)
        return pos + neg

    def compute_output_shape(self, input_shape):
        return input_shape

We will reuse the utility functions from Preso #1 and demonstrate that MonotonicPReLU and InvertedMonotonicPReLU form a mirrored pair and can be used in deep learning.

In [ ]:
def create_model(shape, layer):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=shape))
    model.add(layer)
    model.compile(optimizer='sgd', loss='mse')
    return model

def delta_check(data1, data2, epsilon=1e-2):
    delta = np.abs(data1 - data2)
    return np.max(delta) < epsilon

In [ ]:
forward_layer = MonotonicPReLU()
forward_prelu = create_model((3), forward_layer)
inverted_layer = InvertedMonotonicPReLU(forward_layer)
inverted_prelu = create_model((3), inverted_layer)

data = np.array([[-1, 0, 1]])
forward = forward_prelu.predict(data)
print(forward)
inverted = inverted_prelu.predict(forward)
print(inverted)
print(delta_check(data, inverted))

[[-0.1929388  0.         1.       ]]
[[-1.  0.  1.]]
True


This demonstrates that the MonotonicPRelu/InvertedMonotonicPReLU pair do indeed create a mirrored pair of layers which invert each other's actions.